In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
train_classes = pd.read_csv("/kaggle/input/planets-dataset/planet/planet/train_classes.csv")

In [4]:
train_classes["tag"] = train_classes["tags"].apply(lambda x: sorted(x.split(" "))[-1])

In [5]:
train_labels = {
    row["image_name"]: row["tag"] for i, row in train_classes.iterrows()
}

In [6]:
train_classes.tag.value_counts()

primary              24548
water                 7411
road                  5784
cloudy                2089
selective_logging      291
slash_burn             185
clear                  127
partly_cloudy           23
haze                    12
habitation               7
cultivation              2
Name: tag, dtype: int64

In [7]:
from fastai.vision.all import *

path = Path("/kaggle", "input", "planets-dataset", "planet", "planet", "train-jpg")

In [8]:
def label(file_name):
    return train_labels[file_name.replace(".jpg", "")]
    

In [9]:
label("train_30370.jpg")

'primary'

In [10]:
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    label_func=lambda x: label(x), item_tfms=Resize(224))
    
# if a string is passed into the model argument, it will now use timm (if it is installed)
learn = vision_learner(dls, 'vit_tiny_patch16_224', metrics=error_rate)

learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.668901,0.500663,0.159975,04:23


epoch,train_loss,valid_loss,error_rate,time
0,0.428547,0.390753,0.132057,03:46
